In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, morphology
from skimage.filters import threshold_otsu, sobel
from skimage.segmentation import active_contour
from skimage.measure import label, regionprops
import os

In [2]:
class PolypDetector:
    def __init__(self):
        pass

    def preprocess_image(self, image_path):
        """Preprocess the endoscopic image"""
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not found or path is incorrect")

        # Convert to RGB and resize for consistency
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (512, 512))

        # Apply CLAHE for contrast enhancement
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

        # Denoising
        denoised = cv2.fastNlMeansDenoisingColored(enhanced, None, 10, 10, 7, 21)

        return denoised

    def segment_polyps(self, image):
        """Segment potential polyp regions"""
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 30, 100)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

        # Compatibility fix for OpenCV versions
        contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        filtered_contours = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if area > 500 and 0.7 < circularity < 1.3:
                filtered_contours.append(cnt)

        return filtered_contours

    def highlight_polyps(self, image, contours):
        mask = np.zeros_like(image[:, :, 0])
        cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)
        hulls = [cv2.convexHull(cnt) for cnt in contours]
        hull_mask = np.zeros_like(mask)
        cv2.drawContours(hull_mask, hulls, -1, 255, thickness=cv2.FILLED)

        highlighted = image.copy()
        highlighted[hull_mask == 255] = [255, 0, 0]

        contour_img = image.copy()
        cv2.drawContours(contour_img, hulls, -1, (0, 255, 0), 2)

        return highlighted, contour_img

    def process_image(self, image_path):
        preprocessed = self.preprocess_image(image_path)
        contours = self.segment_polyps(preprocessed)
        highlighted, contour_img = self.highlight_polyps(preprocessed, contours)
        return preprocessed, highlighted, contour_img, len(contours)

def display_results(original, preprocessed, highlighted, contour_img):
    plt.figure(figsize=(20, 10))

    plt.subplot(1, 4, 1)
    plt.imshow(original)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(preprocessed)
    plt.title("Preprocessed Image")
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(highlighted)
    plt.title("Highlighted Polyps")
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(contour_img)
    plt.title("Polyp Contours")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
if __name__ == "__main__":
    image_path = '/content/drive/MyDrive/'

    if not os.path.exists(image_path):
        print(f"Image path does not exist: {image_path}")
    else:
        detector = PolypDetector()
        try:
            original_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
            preprocessed, highlighted, contour_img, num_polyps = detector.process_image(image_path)
            print(f"Detected {num_polyps} potential polyp(s)")
            display_results(original_img, preprocessed, highlighted, contour_img)
        except Exception as e:
            print(f"Error processing image: {e}")